# Search

In [38]:
import json

dummy_data_file_path = '../data/dummy_data.json'

with open(dummy_data_file_path) as dummy_data_file:
    dummy_data = json.load(dummy_data_file)

In [39]:
health_topics = dummy_data['health-topic']
id_and_summary = [(ht['id'], ht['full-summary']) for ht in health_topics]

In [40]:
from sentence_transformers import SentenceTransformer, util

MODEL_NAME = "all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME, cache_folder=".cache_models")
summary_embeddings = model.encode([summary for _, summary in id_and_summary])

In [41]:
query = "cancer"
query_embedding = model.encode(query)

In [42]:
cosine_scores = util.pytorch_cos_sim(query_embedding, summary_embeddings)[0].tolist()

In [43]:
valid_indices = [i for i in range(len(cosine_scores)) if cosine_scores[i] > 0.2]

In [44]:
valid_ids = [ias[0] for i, ias in enumerate(id_and_summary) if i in valid_indices]

In [45]:
valid_hts = [ht for ht in health_topics if ht['id'] in valid_ids]

In [46]:
for ht in valid_hts:
    print(ht['title'])
    print()

Cáncer del ano

